In [1]:
import os, pathlib

file_path = os.path.dirname(os.path.realpath("__file__"))
home_dir = pathlib.Path(file_path).parent.parent
os.chdir(home_dir)

In [2]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
from src.utils import SAVED_PICS_PATH, keras_dataset_image_preprocessing, DENSE_LAYER_NAME, CONV_LAYER_NAME, ALPHAS
from src.utils import SAVED_CLASSIFIER_PATH_MNIST, SAVED_CLADEC_PATH_MNIST, SAVED_REFAE_PATH_MNIST
from src.cladec import ClaDec, RefAE

(train_images, train_labels_c), (test_images, test_labels_c) = keras_dataset_image_preprocessing(keras.datasets.mnist)
SAVED_EVAL_PICS_PATH = SAVED_PICS_PATH.joinpath('fashion_mnist')

tf.random.set_seed(1234)
np.random.seed(1234)

In [3]:
classifier = keras.models.load_model(SAVED_CLASSIFIER_PATH_MNIST)
classifier.evaluate(train_images, train_labels_c)

2022-07-26 21:27:44.462573: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-07-26 21:27:44.509638: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-07-26 21:27:44.509953: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-07-26 21:27:44.510865: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags

1875/1875 [==============================] - 7s 3ms/step - loss: 0.0108 - categorical_accuracy: 0.9964


[0.01077809277921915, 0.9964333176612854]

In [4]:
decoder_dense = ClaDec.create_128_dense_decoder()
for alpha in [0]:
    cladec_path = SAVED_CLADEC_PATH_MNIST.joinpath(f'{alpha:2}')
    cladec_dense = ClaDec.create_from_weights(cladec_path.joinpath(DENSE_LAYER_NAME).joinpath(DENSE_LAYER_NAME),
                                              classifier,
                                              DENSE_LAYER_NAME,
                                              alpha / 100,
                                              decoder_dense)
    print("Evaluating ", cladec_path.joinpath(DENSE_LAYER_NAME))
    cladec_dense.evaluate(train_images, train_labels_c)

Evaluating  /home/tom/Uni_Dev/cladec-mnist/saved-models/mnist/cladec/ 0/my_dense
1875/1875 [==============================] - 10s 5ms/step - loss: 19.9459 - reconstruction_loss: 20.1021 - classification_loss: 0.1035


In [5]:
decoder_conv = ClaDec.create_64_conv_decoder()
for alpha in ALPHAS:
    cladec_path = SAVED_CLADEC_PATH_MNIST.joinpath(f'{alpha:2}')
    cladec_conv = ClaDec.create_from_weights(cladec_path.joinpath(CONV_LAYER_NAME).joinpath(CONV_LAYER_NAME),
                                             classifier,
                                             CONV_LAYER_NAME,
                                             alpha / 100,
                                             decoder_conv)
    print("Evaluating ", cladec_path.joinpath(CONV_LAYER_NAME))
    cladec_conv.evaluate(train_images, train_labels_c)

Evaluating  /home/tom/Uni_Dev/cladec-mnist/saved-models/mnist/cladec/ 0/my_conv
1875/1875 [==============================] - 10s 5ms/step - loss: 1.4234 - reconstruction_loss: 1.4309 - classification_loss: 0.0193
Evaluating  /home/tom/Uni_Dev/cladec-mnist/saved-models/mnist/cladec/ 1/my_conv
1875/1875 [==============================] - 10s 5ms/step - loss: 1.4278 - reconstruction_loss: 1.4499 - classification_loss: 0.0186
Evaluating  /home/tom/Uni_Dev/cladec-mnist/saved-models/mnist/cladec/ 5/my_conv
1875/1875 [==============================] - 10s 5ms/step - loss: 1.3675 - reconstruction_loss: 1.4458 - classification_loss: 0.0177
Evaluating  /home/tom/Uni_Dev/cladec-mnist/saved-models/mnist/cladec/10/my_conv
1875/1875 [==============================] - 10s 5ms/step - loss: 1.2710 - reconstruction_loss: 1.4173 - classification_loss: 0.0177
Evaluating  /home/tom/Uni_Dev/cladec-mnist/saved-models/mnist/cladec/25/my_conv
1875/1875 [==============================] - 10s 5ms/step - loss: 1.

In [6]:
# load RefAE model
refae_dense = RefAE.create_from_weights(SAVED_REFAE_PATH_MNIST.joinpath(DENSE_LAYER_NAME).joinpath(DENSE_LAYER_NAME),
                                        cladec_dense)
refae_conv = RefAE.create_from_weights(SAVED_REFAE_PATH_MNIST.joinpath(CONV_LAYER_NAME).joinpath(CONV_LAYER_NAME),
                                       cladec_conv)
# get reference reconstructions
print("Evaluating ", SAVED_REFAE_PATH_MNIST.joinpath(DENSE_LAYER_NAME))
refae_dense.evaluate(train_images)
print("Evaluating ", SAVED_REFAE_PATH_MNIST.joinpath(CONV_LAYER_NAME))
refae_conv.evaluate(train_images)

Evaluating  /home/tom/Uni_Dev/cladec-mnist/saved-models/mnist/refae/my_dense
1875/1875 [==============================] - 7s 3ms/step - reconstruction_loss: 3.1739
Evaluating  /home/tom/Uni_Dev/cladec-mnist/saved-models/mnist/refae/my_conv
1875/1875 [==============================] - 7s 4ms/step - reconstruction_loss: 0.3530


0.35300254821777344